In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/usr0/home/naveenr/projects/patient_provider')

In [3]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import argparse
import secrets
import json

In [84]:
from patient.simulator import run_multi_seed
from patient.baseline_policies import random_policy, greedy_policy
from patient.online_policies import p_approximation
from patient.offline_policies import offline_solution
from patient.utils import get_save_path, delete_duplicate_results, restrict_resources

In [5]:
is_jupyter = 'ipykernel' in sys.modules

In [126]:
if is_jupyter: 
    seed        = 42
    num_patients = 10
    num_providers = 10
    provider_capacity = 1
    top_choice_prob = 1
    choice_model = "uniform_choice"
    out_folder = "online_baseline"
    exit_option = 0.5
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', help='Random Seed', type=int, default=42)
    parser.add_argument('--n_patients',         '-N', help='Number of patients', type=int, default=100)
    parser.add_argument('--n_patients',         '-N', help='Number of patients', type=int, default=100)
    parser.add_argument('--n_providers',         '-V', help='Number of providers', type=int, default=20)
    parser.add_argument('--top_choice_prob',          help='Probability of picking top choice', type=float, default=0.8)
    parser.add_argument('--provider_capacity',    '-P', help='Provider Capacity', type=int, default=5)
    parser.add_argument('--choice_model', help='Which choice model for patients', type=str, default='uniform_choice')
    parser.add_argument('--exit_option', help='What is the value of the exit option', type=float, default=0.5)
    parser.add_argument('--out_folder', help='Which folder to write results to', type=str, default='online_baseline')

    args = parser.parse_args()

    seed = args.seed
    num_patients = args.n_patients
    num_providers = args.n_providers 
    provider_capacity = args.provider_capacity
    top_choice_prob = args.top_choice_prob
    choice_model = args.choice_model
    exit_option = args.exit_option
    out_folder = args.out_folder

save_name = secrets.token_hex(4)  

In [127]:
results = {}
results['parameters'] = {'seed'      : seed,
        'num_patients'    : num_patients,
        'num_providers': num_providers, 
        'provider_capacity'    : provider_capacity,
        'top_choice_prob': top_choice_prob, 
        'choice_model': choice_model,
        'exit_option': exit_option} 

## Baselines

In [144]:
seed_list = [43,44,45,46,47,48,49,50,51]
restrict_resources()

In [145]:
policy = random_policy
name = "random"

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.mean(rewards['patient_utilities'])/num_patients

Took 0.00041937828063964844 time
Took 0.00038051605224609375 time
Took 0.00038933753967285156 time
Took 0.00038814544677734375 time
Took 0.00037598609924316406 time
Took 0.0003771781921386719 time
Took 0.0003764629364013672 time
Took 0.00037360191345214844 time
Took 0.00037479400634765625 time


0.6350170772320897

In [146]:
policy = greedy_policy
name = "greedy"

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.mean(rewards['patient_utilities'])/num_patients

Took 0.00023412704467773438 time
Took 0.00032711029052734375 time
Took 0.0002033710479736328 time
Took 0.00020074844360351562 time
Took 0.0002009868621826172 time
Took 0.00020241737365722656 time
Took 0.00019931793212890625 time
Took 0.00019884109497070312 time
Took 0.00019812583923339844 time


0.8029755398048316

## Online

In [147]:
policy = p_approximation
name = "p_approximation"

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.mean(rewards['patient_utilities'])/num_patients

Took 0.023489713668823242 time
Took 0.003742694854736328 time
Took 0.003692626953125 time
Took 0.0035479068756103516 time
Took 0.0035843849182128906 time
Took 0.003524303436279297 time
Took 0.0035316944122314453 time
Took 0.0036258697509765625 time
Took 0.003567218780517578 time


0.8700446533989277

## Offline

In [148]:
policy = offline_solution
name = "offline_solution"

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.mean(rewards['patient_utilities'])/num_patients

Took 0.021384239196777344 time
Took 0.0037724971771240234 time
Took 0.003665447235107422 time
Took 0.0035774707794189453 time
Took 0.0036139488220214844 time
Took 0.003551483154296875 time
Took 0.0035696029663085938 time
Took 0.0035712718963623047 time
Took 0.0036215782165527344 time


0.8700446533989277

## Save Data

In [28]:
save_path = get_save_path(out_folder,save_name)

In [29]:
delete_duplicate_results(out_folder,"",results)

In [30]:
json.dump(results,open('../../results/'+save_path,'w'))